<a href="https://colab.research.google.com/github/spdsp04/Telecom_Churn_Prediction_Group_Case_Study/blob/main/Telecom_Chrun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Telecom Chrun Group Case Study

# Telecom Churn Prediction Group Case Study

## Business problem

In the telecommunication industry, customers tend to change operators if not provided with attractive schemes and offers. It is very important for any telecom operator to prevent the present customers from churning to other operators. As a data scientist, your task in this case study would be to build an ML model which can predict if the customer will churn or not in a particular month based on the past data.

So we need to analyse telecom industry data and predict high value customers who are at high risk of churn and identify main indicators of churn

In [13]:
# Supress Warnings

import warnings
warnings.filterwarnings(action="ignore")

In [14]:
# import required Libraries

import pandas as pd # improt pandas
pd.set_option("display.max_columns",None) # for max columns
pd.set_option("display.max_rows",None) # for max rows

import datetime as dt # for date and time

import numpy as np # import numpy

import matplotlib.pyplot as plt # import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec 

import seaborn as sns # import seaborn
sns.set(style="ticks")
%matplotlib inline

from scipy.stats import norm 
from scipy import stats

import sklearn
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import statsmodels.api as sm

In [15]:
# Checking versions of Libraries
f'NumPy version: {np.__version__} | Pandas version: {pd.__version__} | Seaborn version: {sns.__version__}'

'NumPy version: 1.21.5 | Pandas version: 1.3.5 | Seaborn version: 0.11.2'

### Importing Train Dataset and EDA

In [16]:
chrun_train = pd.read_csv('/content/sample_data/Telecom Chrun/train.csv')
print(chrun_train.shape)
chrun_train.head()

(69999, 172)


,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,7.527,48.58,124.38,1.29,32.24,96.68,2.33,0.00,0.0,0.0,0.00,0.0,0.00,2.23,0.00,0.28,5.29,16.04,2.33,0.00,0.00,0.00,0.00,0.00,0.00,7.53,16.04,2.61,46.34,124.38,1.01,18.75,80.61,0.0,0.00,0.0,0.0,0.0,0.0,0.0,65.09,204.99,1.01,0.0,0.0,0.0,8.20,0.63,0.00,0.38,0.0,0.0,81.21,221.68,3.63,2.43,3.68,7.79,0.83,21.08,16.91,0.00,0.00,0.00,3.26,24.76,24.71,0.00,7.61,0.21,7.46,19.96,14.96,0.0,0.0,0.0,0.0,0.0,0.0,7.46,27.58,15.18,11.84,53.04,40.56,0.0,0.0,0.66,0.0,0.0,0.0,1.11,0.69,0.00,3,2,2,77,65,10,65,65,10,6/22/2014,7/10/2014,8/24/2014,65,65,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,42.953,0.00,0.00,0.00,0.00,25.99,30.89,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22.01,29.79,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,30.73,31.66,0.00,0.0,0.0,0.00,30.73,31.66,1.68,19.09,10.53,1.41,18.68,11.09,0.35,1.66,3.40,3.44,39.44,25.03,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,3.44,39.44,25.04,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.00,3,4,5,0,145,50,0,145,50,6/12/2014,7/10/2014,8/26/2014,0,0,0,NaN,7/8/2014,NaN,NaN,1.0,NaN,NaN,145.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,Na

### Meaning of work Churn here what we assume

User who have not done any utilization, either approaching or active - as far as calls, web and so on throughout some undefined time frame. In our investigation we are focusing on high worth users, as around 80% of income comes from the top 20% users. Accordingly, in the event that we can decrease agitate of the great worth users, we will actually want to lessen huge income spillage.

Our dataset is partitioned into 3 stages:
Great stage wherein the client is content with the assistance
Activity pase wherein the user was not content with administration or got a superior proposal from contender and so forth. In this stage, the user for the most part shows unexpected way of behaving in comparison to the great stage. Likewise, it is pivotal to distinguish high-agitate risk user in this stage, so a few remedial moves can be made now to prevet stir
Stir stage wherein the user would have agitated

High-esteem users are the people who have re-energized with a sum more than or equivalent to X, where X is the 70th percentile of the normal re-energize sum in the initial two months (the great stage).
In the wake of distinguishing high worth users, we will label beat users in view of the individuals who have not settled on any decisions (either approaching or cordial) and have not utilized portable web even once in the agitate stage.

How about we initially distinguish high-esteem users

In [17]:
chrun_train['total_data_rech_6'] = chrun_train['total_rech_data_6'] * chrun_train['av_rech_amt_data_6']
chrun_train['total_data_rech_7'] = chrun_train['total_rech_data_7'] * chrun_train['av_rech_amt_data_7']

chrun_train['amt_data_6'] = chrun_train[['total_rech_amt_6','total_data_rech_6']].sum(axis=1)
chrun_train['amt_data_7'] = chrun_train[['total_rech_amt_7','total_data_rech_7']].sum(axis=1)

chrun_train['AVG_amt_data_6_7'] = chrun_train[['amt_data_6','amt_data_7']].mean(axis=1)

In [18]:
chrun_train[['total_rech_amt_6','total_rech_amt_7','AVG_amt_data_6_7']].quantile(np.linspace(.1, 1, 9, 0))

,total_rech_amt_6,total_rech_amt_7,AVG_amt_data_6_7
0.1,0.0,0.0,60.0
0.2,80.0,65.0,108.5
0.3,120.0,110.0,155.0
0.4,170.0,160.0,210.0
0.5,229.0,220.0,275.5
0.6,290.0,284.0,361.0
0.7,378.0,370.0,477.5
0.8,504.0,500.0,656.0
0.9,735.0,730.0,1029.0


According to the meaning of high worth users are those whose re-energize sum ought to be more than or equivalent to 70th percentile.

Be that as it may, assuming we utilize more prominent than or equivalent to, we wind up getting a little above 30K records as against to 29.9K records.
Thus, in the event that we conisder just more prominent than, we get 29.9K records.
Accordingly we have considered more noteworthy than to channel high worth users

In [19]:
high_end_users = chrun_train[(chrun_train['AVG_amt_data_6_7']> chrun_train['AVG_amt_data_6_7'].quantile(0.7))]
high_end_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20998 entries, 4 to 69997
Columns: 177 entries, id to AVG_amt_data_6_7
dtypes: float64(140), int64(28), object(9)
memory usage: 28.5+ MB


### About Chrun Users


Based on last month identify users about to chrun

In [20]:
chrun_train['about_to_chrun_alert'] = np.where(
                            ((chrun_train['total_ic_mou_8'] == 0.00) | (chrun_train['total_og_mou_8'] == 0.00))
                            & ((chrun_train['vol_2g_mb_8'] == 0.00) | (chrun_train['vol_3g_mb_8'] == 0.00))
                            , 1, 0
                           )

In [21]:
print(chrun_train['about_to_chrun_alert'].value_counts())
print(chrun_train['about_to_chrun_alert'].value_counts()/chrun_train.shape[0])

0    62694
1     7305
Name: about_to_chrun_alert, dtype: int64
0    0.895641
1    0.104359
Name: about_to_chrun_alert, dtype: float64


### Remove all features with chrun track of last month

In [22]:
chrun_train = chrun_train.drop(chrun_train.filter(regex='_8', axis = 1).columns, axis=1)
chrun_train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 124 columns):
 #    Column                    Non-Null Count  Dtype  
---   ------                    --------------  -----  
 0    id                        69999 non-null  int64  
 1    circle_id                 69999 non-null  int64  
 2    loc_og_t2o_mou            69297 non-null  float64
 3    std_og_t2o_mou            69297 non-null  float64
 4    loc_ic_t2o_mou            69297 non-null  float64
 5    last_date_of_month_6      69999 non-null  object 
 6    last_date_of_month_7      69600 non-null  object 
 7    arpu_6                    69999 non-null  float64
 8    arpu_7                    69999 non-null  float64
 9    onnet_mou_6               67231 non-null  float64
 10   onnet_mou_7               67312 non-null  float64
 11   offnet_mou_6              67231 non-null  float64
 12   offnet_mou_7              67312 non-null  float64
 13   roam_ic_mou_6             67231 non-null  fl

In [23]:
#to Check null and unique values in parallel
null_unique = pd.DataFrame()
null_unique['nulls'] = pd.Series(chrun_train.isnull().sum())
null_unique['unique'] = pd.Series(chrun_train.nunique())
null_unique.transpose()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,arpu_6,arpu_7,onnet_mou_6,onnet_mou_7,offnet_mou_6,offnet_mou_7,roam_ic_mou_6,roam_ic_mou_7,roam_og_mou_6,roam_og_mou_7,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_mou_6,loc_og_mou_7,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_mou_6,std_og_mou_7,isd_og_mou_6,isd_og_mou_7,spl_og_mou_6,spl_og_mou_7,og_others_6,og_others_7,total_og_mou_6,total_og_mou_7,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_mou_6,loc_ic_mou_7,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_mou_6,std_ic_mou_7,total_ic_mou_6,total_ic_mou_7,spl_ic_mou_6,spl_ic_mou_7,isd_ic_mou_6,isd_ic_mou_7,ic_others_6,ic_others_7,total_rech_num_6,total_rech_num_7,total_rech_amt_6,total_rech_amt_7,max_rech_amt_6,max_rech_amt_7,date_of_last_rech_6,date_of_last_rech_7,last_day_rch_amt_6,last_day_rch_amt_7,date_of_last_rech_data_6,date_of_last_rech_data_7,total_rech_data_6,total_rech_data_7,max_rech_data_6,max_rech_data_7,count_rech_2g_6,count_rech_2g_7,count_rech_3g_6,count_rech_3g_7,av_rech_amt_data_6,av_rech_amt_data_7,vol_2g_mb_6,vol_2g_mb_7,vol_3g_mb_6,vol_3g_mb_7,arpu_3g_6,arpu_3g_7,arpu_2g_6,arpu_2g_7,night_pck_user_6,night_pck_user_7,monthly_2g_6,monthly_2g_7,sachet_2g_6,sachet_2g_7,monthly_3g_6,monthly_3g_7,sachet_3g_6,sachet_3g_7,fb_user_6,fb_user_7,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability,total_data_rech_6,total_data_rech_7,amt_data_6,amt_data_7,AVG_amt_data_6_7,about_to_chrun_alert
nulls,0,0,702,702,702,0,399,0,0,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,0,0,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,2768,2687,0,0,2768,2687,2768,2687,2768,2687,0,0,0,0,0,0,1101,1234,0,0,52431,52134,52431,52134,52431,52134,52431,52134,52431,52134,52431,52134,0,0,0,0,52431,52134,52431,52134,52431,52134,0,0,0,0,0,0,0,0,52431,52134,0,0,0,0,0,52431,52134,0,0,0,0
unique,69999,1,1,1,1,1,1,61615,61425,20057,20076,26254,26139,5306,4240,6568,5305,11491,11359,17848,17641,3376,3365,1945,2103,22309,22012,14554,14745,16059,16234,2074,1991,1,1,21837,22088,1103,1084,3503,3827,914,148,33135,33195,11637,11595,19034,18945,6200,6287,24442,24258,5323,5479,7931,8033,2624,2695,1,1,9876,10112,27432,27418,78,103,4421,4640,1523,1658,94,96,2108,2147,190,178,30,31,179,166,30,31,36,41,47,45,30,33,23,27,792,837,11494,11443,9768,10296,5506,5389,5389,5056,2,2,5,6,30,33,10,13,23,26,2,2,3455,10609,10257,9617,2,1397,1489,3419,3604,5453,2


In [24]:
# Checking missing values in dataset in percentage with decreseing order
pd.options.display.float_format = '{:.2f}'.format # to display values up to 02 decimal places
missing_values = pd.DataFrame((chrun_train.isnull().sum()/len(chrun_train))*100).reset_index().rename(columns = {'index': 'feature', 0: 'Missing%'}).sort_values('Missing%',ascending = False)
missing_values.transpose()

,101,89,118,81,83,85,87,111,99,91,97,82,100,119,112,90,98,84,86,92,88,102,53,41,45,47,49,65,55,57,59,61,67,69,37,39,51,29,13,21,25,19,27,17,15,23,31,33,11,9,35,14,12,22,16,18,10,70,38,68,66,36,20,50,52,40,34,42,32,46,30,48,28,62,54,26,24,58,56,60,78,77,2,3,4,6,107,122,103,104,105,106,121,113,120,117,110,109,108,114,115,116,0,96,71,5,7,8,43,44,1,63,64,72,95,73,74,75,76,79,80,93,94,123
feature,night_pck_user_6,count_rech_3g_6,total_data_rech_6,date_of_last_rech_data_6,total_rech_data_6,max_rech_data_6,count_rech_2g_6,fb_user_6,arpu_2g_6,av_rech_amt_data_6,arpu_3g_6,date_of_last_rech_data_7,arpu_2g_7,total_data_rech_7,fb_user_7,count_rech_3g_7,arpu_3g_7,total_rech_data_7,max_rech_data_7,av_rech_amt_data_7,count_rech_2g_7,night_pck_user_7,std_ic_t2t_mou_6,og_others_6,loc_ic_t2t_mou_6,loc_ic_t2m_mou_6,loc_ic_t2f_mou_6,spl_ic_mou_6,std_ic_t2m_mou_6,std_ic_t2f_mou_6,std_ic_t2o_mou_6,std_ic_mou_6,isd_ic_mou_6,ic_others_6,isd_og_mou_6,spl_og_mou_6,loc_ic_mou_6,std_og_t2m_mou_6,roam_ic_mou_6,loc_og_t2f_mou_6,loc_og_mou_6,loc_og_t2m_mou_6,std_og_t2t_mou_6,loc_og_t2t_mou_6,roam_og_mou_6,loc_og_t2c_mou_6,std_og_t2f_mou_6,std_og_t2c_mou_6,offnet_mou_6,onnet_mou_6,std_og_mou_6,roam_ic_mou_7,offnet_mou_7,loc_og_t2f_mou_7,roam_og_mou_7,loc_og_t2t_mou_7,onnet_mou_7,ic_others_7,isd_og_mou_7,isd_ic_mou_7,spl_ic_mou_7,std_og_mou_7,loc_og_t2m_mou_7,loc_ic_t2f_mou_7,loc_ic_mou_7,spl_og_mou_7,std_og_t2c_mou_7,og_others_7,std_og_t2f_mou_7,loc_ic_t2t_mou_7,std_og_t2m_mou_7,loc_ic_t2m_mou_7,std_og_t2t_mou_7,std_ic_mou_7,std_ic_t2t_mou_7,loc_og_mou_7,loc_og_t2c_mou_7,std_ic_t2f_mou_7,std_ic_t2m_mou_7,std_ic_t2o_mou_7,date_of_last_rech_7,date_of_last_rech_6,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_7,monthly_3g_6,AVG_amt_data_6_7,monthly_2g_6,monthly_2g_7,sachet_2g_6,sachet_2g_7,amt_data_7,aon,amt_data_6,churn_probability,sachet_3g_7,sachet_3g_6,monthly_3g_7,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,id,vol_3g_mb_7,total_rech_num_6,last_date_of_month_6,arpu_6,arpu_7,total_og_mou_6,total_og_mou_7,circle_id,total_ic_mou_6,total_ic_mou_7,total_rech_num_7,vol_3g_mb_6,total_rech_amt_6,total_rech_amt_7,max_rech_amt_6,max_rech_amt_7,last_day_rch_amt_6,last_day_rch_amt_7,vol_2g_mb_6,vol_2g_mb_7,about_to_chrun_alert
Missing%,74.90,74.90,74.90,74.90,74.90,74.90,74.90,74.90,74.90,74.90,74.90,74.48,74.48,74.48,74.48,74.48,74.48,74.48,74.48,74.48,74.48,74.48,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.95,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,1.76,1.57,1.00,1.00,1.00,0.57,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# New Section